In [1]:
import pandas as pd

In [21]:
pd.options.plotting.backend = "plotly"

In [61]:
def preporcessing():
    part1 = pd.read_csv("/home/amait/Documents/fac/series_chrono/projet/data/adrien_part_1.csv")
    part2 = pd.read_csv("/home/amait/Documents/fac/series_chrono/projet/data/adrien_part_2.csv")
    data = pd.concat([part1, part2])                                                                # concat des 2 périodes
    data = data.astype('datetime64[ns, Europe/Paris]')
    data["paid"] = data.deliveryTime.dt.microsecond != 0                                            # connaissance métier
    data.drop(["deliveryTime", "cookTime"], axis=1, inplace=True)                                   # drop useless columns
    data.set_index("orderedAt", inplace=True)                                                       # set as index orderedAt 
    data = data[data.paid==True].resample("1d").count()                                             # resample by days and count orders
    data.paid = data.paid / 5
    data.columns = ["n_orders"]
    data.index.name = "ds"
    data.to_pickle("data/orders.csv")                                                                  # save DataFrame
    
preporcessing()

In [62]:
ts = pd.read_pickle("data/orders.csv")
ts.plot(title=f'Nombre de commandes par jour de {ts.index[0].strftime("%d-%m-%Y")} au {ts.index[-1].strftime("%d-%m-%Y")}')

La série décrit le nombre de commandes livrées par jour du 02 janvier 2020 au 15 octobre 2021 soit jours. On observe une forte saisonalité sur les jours de la semaine. Une modélisation par un modèle SARIMA peut être envisagé.

In [58]:
ts["weekday"] = ts.index.weekday
order_by_weekday = ts.groupby("weekday").mean()
order_by_weekday.plot()

Cependant, il est clair que la série n'est pas stationnaire car :
- il semble y avoir une tendance légèrement à la hausse.
- la série ne semble pas avoir une variance constante au cours du temps.

Il faut alors dans un premier temps corrigé ce problème d'hétéroscédasticité. Plusieurs transformation existe pour y parvenir :
- passage au log
- Box-Cox
- autres ?